In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

%matplotlib inline

In [3]:
#Tasks.Imagine that you got a job as an analyst in the advertising department, and your first task is to help colleagues sort out some issues:

#Get statistics by day. Calculate the number of all events by day, the number of impressions, the number of clicks, the number of unique ads and unique campaigns.
#Figure out why there was such a jump on 2019-04-05? What events have become more frequent? Do all ads or just some?
#Find the top 10 ads by CTR for all time. CTR is the ratio of all ad clicks to views. For example, if an ad had 100 impressions and 2 clicks, CTR = 0.02. Does the average and median CTR of ads differ in our data?
#It looks like there is a bug in our logs, ads come with clicks, but without impressions! How many such ads are there, are there any patterns? Is this problem observed on all platforms?
#Are there any differences in CTR between ads with and without video? And what is the 95th percentile CTR for all ads for 2019-04-04?
#For the financial report, we need to calculate our earnings by day. On which day did we earn the most? Which one is less? We charge customers money if there was a click on a CPC ad, and we charge money for each CPM ad impression, if the CPM ad has a price of 200 rubles, then we earn 200 / 1000 for one impression.
#Which platform is the most popular for placing advertisements? How many percent of impressions are on each of the platforms (platform column)?
#And are there any ads that were clicked on first and only then shown?
   
#Description of the columns
#date — the day on which the events take place
#time — the exact time of the event
#event — the type of event, it can be either an impression or a click on an advertisement
#platform — the platform on which the advertising event took place
#ad_id — the ad ID
#client_union_id — the ID of the advertising client
#campaign_union_id — the ID of the advertising campaign
#ad_cost_type is the type of ad with pay per click (CPC) or per impressions (CPM)
#ad_cost is the cost of the ad in rubles, for CPC ads it is the cost per click, for CPM it is the price per 1000 impressions
#has_video — does the ad have a video
#target_audience_count — the size of the audience the ad is targeting

In [ ]:
ads_data=pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-m-nurdaulet/ads_data.csv.zip', compression ='zip')

In [ ]:
ads_data.head()

In [ ]:
#Get statistics by day. Calculate the number of all events by day, the number of impressions, the number of clicks, the number of unique ads and unique campaigns.
#Figure out why there was such a jump on 2019-04-05? What events have become more frequent? For all ads or only for

In [ ]:
ads_data['full_date']=pd.to_datetime(ads_data.time, unit='s')

# Convert timestamps in 'time' column to datetime format and assign to 'full_date' column.


In [ ]:
ads_data['date']=pd.to_datetime(ads_data.date)
# Convert 'date' column to datetime format using 'to_datetime' function from pandas library.


In [ ]:

ads_data.head()

In [ ]:
ads_data.shape

In [ ]:
ads_data.dtypes

In [ ]:
ads_data.full_date.dt.day

In [ ]:
ads_data.groupby('date') \
   .agg({'ad_id':'count'}).plot()

# Group the data by 'date' and aggregate the count of 'ad_id' for each date.
# Then, create a plot of the counts over time.


In [ ]:
ads_data.groupby(['date','event'],as_index=False) \
   .agg({'ad_id':'count'})\
   .pivot(index='date', columns='event', values='ad_id').reset_index()

# Group the data by 'date' and 'event', aggregating the count of 'ad_id' for each combination.
# Pivot the resulting DataFrame to reshape it so that each event type becomes a separate column.
# Reset the index to make 'date' a regular column instead of an index.


In [ ]:
??pd.pivot

In [ ]:
ads_data[ads_data.date == '2019-04-05'] \
   .groupby('ad_id')\
   .agg({'time': 'count'}) \
   .sort_values('time',ascending = False)

# Filter the data for records with the date '2019-04-05'.
# Group the filtered data by 'ad_id' and aggregate the count of 'time' for each ad.
# Sort the aggregated counts of 'time' in descending order to identify the most frequent ads on that date.


In [ ]:
ad_id =112583

In [ ]:
ads_data.query('ad_id == @ad_id') \
   .groupby('date') \
   .agg({'time':'count'}) \

# Filter the data for records where the 'ad_id' matches the specified value.
# Group the filtered data by 'date' and aggregate the count of 'time' for each date.


In [ ]:
ads_data.query('ad_id == @ad_id').head(1)

# Filter the data for records where the 'ad_id' matches the specified value, and retrieve the first record.


In [ ]:
#Find the top 10 ads by CTR for all time. CTR is the ratio of all ad clicks to views. For example, if an ad had 100 impressions and 2 clicks, CTR = 0.02. Does the average and median CTR of ads differ in our data?

In [ ]:

ads_data_by_ad = ads_data.groupby(['ad_id','event'],as_index=False) \
   .agg({'time':'count'})\
   .pivot(index='ad_id', columns='event', values='time').reset_index()

# Group the data by 'ad_id' and 'event', aggregating the count of 'time' for each combination.
# Pivot the resulting DataFrame to reshape it so that each event type becomes a separate column.
# Reset the index to make 'ad_id' a regular column instead of an index.


In [ ]:
ads_data_by_ad= ads_data_by_ad.assign(ctr = ads_data_by_ad.click / ads_data_by_ad.view,
                         ctr_per=100 * ads_data_by_ad.click/ ads_data_by_ad.view)

# Calculate the Click-Through Rate (CTR) and CTR percentage for each ad by dividing the number of clicks by the number of views.
# Assign the calculated CTR and CTR percentage as new columns 'ctr' and 'ctr_per' respectively in the 'ads_data_by_ad' DataFrame.


In [ ]:
ads_data_by_ad.head()

In [ ]:
ads_data_by_ad.sort_values('ctr', ascending = False).head(10)

# Sort the 'ads_data_by_ad' DataFrame by the calculated CTR in descending order to find the top 10 ads with the highest CTR.


In [ ]:
ads_data_by_ad.ctr.mean()

In [ ]:
ads_data_by_ad.ctr.median()

In [ ]:
sns.displot(ads_data_by_ad.ctr, kde= False)

In [ ]:
#It looks like there is a bug in our logs, ads come with clicks, but without impressions! How many such ads are there, are there any patterns? Is this problem observed on all platforms?

In [ ]:
ads_data_by_ad=ads_data_by_ad.fillna(0)

In [ ]:
ads_ids_bug = ads_data_by_ad.query('click == 0')
# Filter the 'ads_data_by_ad' DataFrame to select ads where clicks are zero, indicating the presence of clicks without impressions.
# This identifies the number of ads affected by the bug.


In [ ]:
ads_ids_bug = ads_data_by_ad.query('click == 0').ad_id

# Filter the 'ads_data_by_ad' DataFrame to select ad IDs where clicks are zero, indicating the presence of clicks without impressions.


In [ ]:
ads_data[ads_data.ad_id.isin(ads_ids_bug)]  \
   .groupby('platform',as_index = False) \
   .agg({'ad_id': 'count'})\
   .sort_values('ad_id')

# Filter the 'ads_data' DataFrame to select records corresponding to ad IDs affected by the bug.
# Group the filtered data by 'platform', aggregating the count of affected ad IDs for each platform.
# Sort the resulting DataFrame by the count of affected ad IDs in ascending order.


In [ ]:
ads_data \
   .groupby('platform',as_index = False) \
   .agg({'ad_id': 'count'})\
   .sort_values('ad_id')

# Group the 'ads_data' DataFrame by 'platform', aggregating the count of ad IDs for each platform.
# Sort the resulting DataFrame by the count of ad IDs in ascending order.


In [ ]:
#Are there any differences in CTR between ads with and without video? And what is the 95th percentile CTR for all ads for 2019-04-04?

In [ ]:
df_2019_04_04 =ads_data[ads_data.date == '2019-04-04'].groupby(['ad_id','event'],as_index=False) \
   .agg({'time':'count'})\
   .pivot(index='ad_id', columns='event', values='time').reset_index() \
   .fillna(0)

# Filter the 'ads_data' DataFrame to select records for the date '2019-04-04'.
# Group the filtered data by 'ad_id' and 'event', aggregating the count of 'time' for each combination.
# Pivot the resulting DataFrame to reshape it so that each event type becomes a separate column.
# Reset the index to make 'ad_id' a regular column instead of an index.
# Fill NaN values with zero.


In [ ]:
df_2019_04_04['ctr'] = df_2019_04_04.click / df_2019_04_04.view

# Calculate the Click-Through Rate (CTR) for each ad on the date '2019-04-04' by dividing the number of clicks by the number of views.
# Assign the calculated CTR as a new column 'ctr' in the 'df_2019_04_04' DataFrame.

                         

In [ ]:
df_2019_04_04[df_2019_04_04.click>0].ctr.quantile(q=0.95)

# Calculate the 95th percentile Click-Through Rate (CTR) for ads with at least one click on the date '2019-04-04'.


In [ ]:
#For the financial report, we need to calculate our earnings by day. On which day did we earn the most? Which one is less? We charge customers money if there was a click on a CPC ad, and we charge money for each CPM ad impression, if the CPM ad has a price of 200 rubles, then we earn 200 / 1000 for one impression.
#Which platform is the most popular for placing advertisements? How many percent of impressions are on each of the platforms (platform column)?

In [ ]:
ads_data[(ads_data.ad_cost_type == 'CPC') & (ads_data.event == 'click')].ad_cost.sum()

# Filter the 'ads_data' DataFrame to select records for CPC ads where events are clicks,
# then sum up the total cost of these clicks.


In [ ]:
ads_data[(ads_data.ad_cost_type == 'CPM') & (ads_data.event == 'view')].ad_cost.sum() /1000

# Filter the 'ads_data' DataFrame to select records for CPM ads where events are views,
# then sum up the total cost of these views, considering that CPM cost is per thousand impressions (divide by 1000).


In [ ]:
# Are there any ads that were clicked on first and only then shown?

In [ ]:
ads_data.groupby(['ad_id','event'],as_index =False) \
   .agg({'time':'min'}) \
   .pivot(index='ad_id', columns='event', values='time').reset_index() \
   .query('click<view')

# Group the data by 'ad_id' and 'event', aggregating the minimum 'time' for each combination.
# Pivot the resulting DataFrame to reshape it so that each event type becomes a separate column.
# Reset the index to make 'ad_id' a regular column instead of an index.
# Filter the DataFrame to select records where the click event occurred before the view event for the same ad.


In [ ]:
ads_data[ads_data.ad_id == 16548].sort_values('time')

# Filter the 'ads_data' DataFrame to select records for a specific ad (ad_id = 16548) and sort them by 'time'.
